<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code streams real-time option quotes using the ThetaData API to calculate and print straddle prices for a specific option contract. It defines a callback function to handle incoming quote messages and update the bid, ask, and mid prices of the straddle. The main function initializes the ThetaClient, connects to the stream, and requests quote updates for both call and put options. This setup is useful for traders and analysts monitoring option prices and straddle strategies in real-time.

In [ ]:
import datetime as dt

In [ ]:
import thetadata.client
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType
)
from thetadata import StreamResponseType as ResponseType

Initialize last call and put quotes and price variable

In [ ]:
last_call_quote = Quote()
last_put_quote = Quote()
price = 0

Callback function to handle incoming stream messages and update straddle prices

In [ ]:
def callback_straddle(msg):
    """Handles incoming stream messages to update straddle prices.
    
    Parameters
    ----------
    msg : StreamMsg
        Incoming stream message containing quote data.
    
    Returns
    -------
    None
    """

    if (msg.type != StreamMsgType.QUOTE):
        return

    if msg.contract.isCall:
        last_call_quote.copy_from(msg.quote)
    else:
        last_put_quote.copy_from(msg.quote)

    straddle_bid = round(last_call_quote.bid_price + last_put_quote.bid_price, 2)
    straddle_ask = round(last_call_quote.ask_price + last_put_quote.ask_price, 2)
    straddle_mid = round((straddle_bid + straddle_ask) / 2, 2)
    
    time_stamp = thetadata.client.ms_to_time(
        msg.quote.ms_of_day
    )

    if price != straddle_mid:
        print(
            f"time: {time_stamp} bid: {straddle_bid} mid: {straddle_mid} ask: {straddle_ask}"
        )
        price = straddle_mid

Main function to set up streaming for straddle prices

In [ ]:
def streaming_straddle():
    """Streams option quotes to calculate straddle prices in real-time.
    
    Returns
    -------
    None
    """
    
    client = ThetaClient(
        username="strimp101@gmail.com",
        passwd="kdk_fzu6pyb0UZA-yuz"
    )

    client.connect_stream(
        callback_straddle
    )
    
    req_id_call = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.CALL
    )  # Request quote updates
    
    req_id_put = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.PUT
    )

    if (
        client.verify(req_id_call) != ResponseType.SUBSCRIBED
        or client.verify(req_id_put) != ResponseType.SUBSCRIBED
    ):
        raise Exception(
            "Unable to stream contract. A standard/PRO subscription required."
        )

Call the main function to start streaming

In [ ]:
streaming_straddle()

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.